<a href="https://colab.research.google.com/github/Facusc16/ufc_web_scraper/blob/main/ufc_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requerimientos

## Librerías

In [ ]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

from google.colab import drive
from IPython.display import clear_output

from datetime import date
import time

## Conectar con Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Definir Scrapers

## Setup de las request: headers, cookies, y params

In [ ]:
config = {
    'fighters': {
        'url': 'https://www.ufc.com/views/ajax',  # URL para solicitar los datos de todos los peleadores vía AJAX
        'headers': {  # Encabezados HTTP para simular una solicitud del navegador
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 OPR/120.0.0.0',
            'Referer': 'https://www.ufc.com/',
            'Origin': 'https://www.ufc.com',
            'X-Requested_With': 'XMLHttpRequest',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Languege': 'es-ES,es;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br, zstd'
        },
        'cookies': {  # Cookies para mantener la sesión y evitar bloqueos del servidor
            "OptanonAlertBoxClosed": "2025-07-19T23:25:31.501Z",
            "_gcl_au": "1.1.1845236352.1754711655",
            "STYXKEY_region": "LATIN_AMERICA.AR.en.Default",
            "__gads": "ID=8bdcf0954598d951:T=1752967439:RT=1755202417:S=ALNI_MYRDIjteUuPeFLf5qk0ZSaEzKXXeg",
            "__gpi": "UID=000010e959d7d8a7:T=1752967439:RT=1755202417:S=ALNI_Mbherse1w10CRYjib8OAX2IAJzpMQ",
            "__eoi": "ID=4f7dc6fb420c1067:T=1752967439:RT=1755202417:S=AA-AfjZsnOPoVSUNo2bh-G8tDHl-",
            "OptanonConsent": "isGpcEnabled=0&datestamp=Thu+Aug+14+2025+17%3A13%3A44+GMT-0300+(hora+est%C3%A1ndar+de+Argentina)&version=202411.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=c3a37c83-ec1a-4985-a557-3bde887fc056&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=1%3A1%2C2%3A0%2C3%3A0%2C4%3A0%2C5%3A0&AwaitingReconsent=false&intType=3&geolocation=%3B"
        },
        'params': {  # Parámetros requeridos por el backend de la UFC para devolver los peleadores
            'view_name': 'all_athletes',
            'view_display_id': 'page',
            'view_args': '',
            'view_path': '/athletes/all',
            'view_base_path': '',
            'view_dom_id': '2c3c312fe4dc9787715f2863a8c5a6fa87f2e1be4cf3c18799fea36d0b19d864',
            'pager_element': 0,
            'page': 0,
            'ajax_page_state[theme]': 'ufc',
            'ajax_page_state[theme_token]': '',
            'ajax_page_state[libraries]': 'eJx1UkGOwyAM_FAIT0IOOIlbgiPstE1fvwSodg9bCaGZwRjbA4SgDOm00ME4Z046TODvTrms3f7B7iZfjxRf5R6qYnb42lkwuJlioWLhKEFyTBt9DVkwYYY4hAfaBwXkYQaPKjbkY4c4NjZGSnfzpLCg_h_wIHyKgRu8hggnH-oCiecH5tNyQs9x2DnGz60LmyupDHKK4lZ6EhyO2dvpUOUkFbcqjWBEr1VZIk8QjRf5S2-N1YQmZN4DP1OXRCktBnSNqO2FDdNhNqAWscPSZCntT5DNTFm0K4pmZi41_PIVIXT-xEkzpiCGd6WN3jWRgyA2lxlPkFKLdH1mgpD9Wgapa-n-sqUe0rLqDlINWznTG5S4Vucie4hdKEmXj94y2V84Qoyud1mn4c7yM_qLpZQ8VIts3ceNwxGxSe5yzRVn7Qd0ndJMqbTsxOfLu-bxRzVN_QFZ8BWy'
        }
    },
    'events': {
        'url': 'https://www.ufc.com/views/ajax',  # URL para solicitar datos de eventos pasados de la UFC vía AJAX
        'headers': {  # Encabezados HTTP para simular una solicitud del navegador
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 OPR/120.0.0.0',
            'Referer': 'https://www.ufc.com/',
            'Origin': 'https://www.ufc.com',
            'X-Requested_With': 'XMLHttpRequest',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Languege': 'es-ES,es;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br, zstd'
        },
        'cookies': {  # Cookies para mantener la sesión y evitar bloqueos del servidor
            "OptanonAlertBoxClosed": "2025-07-19T23:25:31.501Z",
            "_gcl_au": "1.1.1845236352.1754711655",
            "STYXKEY_region": "LATIN_AMERICA.AR.en.Default",
            "__gads": "ID=8bdcf0954598d951:T=1752967439:RT=1755201404:S=ALNI_MYRDIjteUuPeFLf5qk0ZSaEzKXXeg",
            "__gpi": "UID=000010e959d7d8a7:T=1752967439:RT=1755201404:S=ALNI_Mbherse1w10CRYjib8OAX2IAJzpMQ",
            "__eoi": "ID=4f7dc6fb420c1067:T=1752967439:RT=1755201404:S=AA-AfjZsnOPoVSUNo2bh-G8tDHl-",
            "OptanonConsent": "isGpcEnabled=0&datestamp=Thu+Aug+14+2025+16%3A57%3A26+GMT-0300+(hora+est%C3%A1ndar+de+Argentina)&version=202411.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=c3a37c83-ec1a-4985-a557-3bde887fc056&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=1%3A1%2C2%3A0%2C3%3A0%2C4%3A0%2C5%3A0&AwaitingReconsent=false&intType=3&geolocation=%3B"
        },
        'params': {  # Parámetros requeridos por el backend de la UFC para devolver la información de los eventos
            'view_name': 'events_upcoming_past',
            'view_display_id': 'past',
            'view_args': '',
            'view_path': '/events',
            'view_base_path': '',
            'view_dom_id': '2e422ed07970c73b149fbe6e6a2a328005c2f81634d5e5add883f14b4dab4ab4',
            'pager_element': 0,
            'page': 0,
            'ajax_page_state[theme]': 'ufc',
            'ajax_page_state[theme_token]': '',
            'ajax_page_state[libraries]': 'eJx1kQGOhCAMRS8kcpG9A6lQlBmkhhYd9_TLgGYn2WxC4P_XpC0tOCcE6dRwidFnSjJMYJ9GqJ5Nf2jz4H9Dgi8Z3I56Dw5p8AGjM3OmsmmMuGKScaEcvmt6iEZg4iHCSUWMC2xpx3xqSmgpDhvFqF0uG8TxrVUM6ckDnyy41qKMQ_FWT0WEEjdtITuFe61yWyqMsRkfomBW1aLt4TnSBFFZ5k_76G7BTF3QoYTUAWKXj9x_sWrfbMGQFFgJlJprfSuXaXN0dFQHUdQKobsN5v4XrkObICsfMstFBJUnqq3_-gXBXb5NsAoDjnVGZyZIqcdM64p1f75Cz2h8mBfZgFlDkbYLuDs1kSzEC9Rs883PulXNCNkutUAe9oAH63aPK7kSsSMDD3iZGUXf4uIh-ZBq64Ztfq-1UXVT1ekPOsf5fw'
        }
    }
}

## Profiles

### Funciones

In [ ]:
def data_filler(soup, class_):
  tag = soup.find(class_=class_)
  return tag.text.strip() if tag else None

In [ ]:
def safe_int(soup, id):
  el = soup.find(id=id)
  return int(el.text) if el and el.text.isdigit() else None

In [ ]:
def get_tags(soup, keys, div, label_class, value_class):
  data = {key: None for key in keys}

  values = [value for value in soup.find_all(class_=div)]

  for value in values:
    label_tag = value.find(class_=label_class).text.strip().lower().replace(".", "").replace(" ", "_")
    value_tag = value.find(class_=value_class).text.split("(")[0].strip()

    try:
      if value_tag == "":
        value_tag = None
      elif "." in value_tag:
        value_tag = float(value_tag)
      else:
        value_tag = int(value_tag)
    except ValueError:
      pass

    data[label_tag] = value_tag

  return data

### Profiles Scraper

In [ ]:
def get_fighter_profile(url, headers):

  try:

    # Descargar y analizar el código HTML de la página
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')

    # Extraer biografía y estadísticas del peleador
    method_position = get_tags(soup, ['ko/tko', 'dec', 'sub', 'standing', 'clinch', 'ground'], "c-stat-3bar__group", "c-stat-3bar__label", "c-stat-3bar__value")
    strikes_takedowns = get_tags(soup, ["sig_strikes_landed", "sig_strikes_attempted", "takedowns_landed", "takedowns_attempted"], "c-overlap__stats", "c-overlap__stats-text", "c-overlap__stats-value")
    bio = get_tags(soup, ['place_of_birth', 'fighting_style', 'age', 'height', 'weight', 'octagon_debut','reach', 'leg_reach'], "c-bio__field", "c-bio__label", "c-bio__text")

    # Completar información del peleador
    fighter_profile = {
      'name': data_filler(soup, "hero-profile__name"),  # Biografía
      'nickname': data_filler(soup, "hero-profile__nickname"),
      'place_of_birth': bio['place_of_birth'],
      'age': bio['age'],
      'height': bio['height'],  # Medidas
      'weight': bio['weight'],
      'reach': bio['reach'],
      'leg_reach': bio['leg_reach'],
      'fighting_style': bio['fighting_style'],  # Información de carrera
      'ufc_debut_date': bio['octagon_debut'],
      'weigh_class': data_filler(soup, "hero-profile__division-title"),
      'professional_record': data_filler(soup, "hero-profile__division-body"),
      'ko_tko_wins': method_position['ko/tko'],
      'decision_wins': method_position['dec'],
      'submission_wins': method_position['sub'],
      'significant_strikes_landed': strikes_takedowns['sig_strikes_landed'],  # Estadísticas de pelea
      'significant_strikes_attempted': strikes_takedowns['sig_strikes_attempted'],
      'strikes_by_standing': method_position['standing'],
      'strikes_by_clinch': method_position['clinch'],
      'strikes_by_ground': method_position['ground'],
      'strikes_to_the_head': safe_int(soup, 'e-stat-body_x5F__x5F_head_value'),
      'strikes_to_the_body': safe_int(soup, 'e-stat-body_x5F__x5F_body_value'),
      'strikes_to_the_leg': safe_int(soup, 'e-stat-body_x5F__x5F_leg_value'),
      'takedowns_landed': strikes_takedowns['takedowns_landed'],
      'takedowns_attempted': strikes_takedowns['takedowns_attempted'],
      'is_active': True if soup.find(class_="hero-profile__tag") else False  # Estado de actividad
    }
    return fighter_profile

  except AttributeError:
    return None

### Fighters Scraper

In [ ]:
def ufc_fighters_scraper(url, headers, cookies, params):
  fighters_profiles_list = []
  count = 0

  while True:

    # Descargar y convertir la respuesta JSON de la página
    response = requests.get(url, headers=headers, cookies=cookies, params=params)
    data = response.json()

    # Extraer contenido HTML
    html = None
    for item in data:
      if item.get('command') == 'insert' and item.get('data'):
        html = item['data']
        break

    # Analizar  HTML y obtener enlaces
    soup = BeautifulSoup(html, 'html.parser')
    href_in_page = soup.find_all('a', class_='e-button--black')

    if not href_in_page:
      break

    # Obtener y guardar perfil de los peleadores
    for a_tag in href_in_page:
      fighter_profile = get_fighter_profile('https://www.ufc.com' + a_tag['href'], headers)
      if fighter_profile is not None:
        fighters_profiles_list.append(fighter_profile)
      count +=1

    # Imprimir estado del progreso de la ejecución
    print(f"Registrados {len(fighters_profiles_list)} peleadores")
    clear_output(wait=True)

    params['page'] += 1
    time.sleep(1)

  return pd.DataFrame(fighters_profiles_list)

## Events

### Event scraper

In [ ]:
def get_event(url, headers):

  # Descargar y analizar el código HTML de la página
  html = requests.get(url, headers=headers)
  soup = BeautifulSoup(html.content, 'html.parser')

  # Buscar las IDs de cada pelea
  tags = soup.find_all('div', class_='c-listing-fight')

  if not tags:
    return None

  tag_id = tags[0].get('data-fmid')
  if not tag_id:
    return None

  # Definir url para la petición de la pelea
  event_url = f"https://d29dxerjsp82wz.cloudfront.net/api/v3/fight/live/{tag_id}.json"

  # Obtener información de la pelea
  res = requests.get(event_url, headers=headers)
  data = res.json()

  data = data['LiveFightDetail']

  # Completar informción de la pelea
  event_data = data['Event']
  event = {
    'name': event_data['Name'],
    'date': event_data['StartTime'],
    'time_zone': event_data['TimeZone'],
    'country': event_data['Location']['Country'],
    'state': event_data['Location']['State'],
    'city': event_data['Location']['City'],
    'venue': event_data['Location']['Venue']
  }

  return event

### Fights scraper

In [ ]:
def get_event_fights(url, headers):

  def get_outcome(data):
    outcomes = [fighter.get('Outcome', {}).get('OutcomeId') for fighter in data if fighter.get('Outcome')]

    if 4 in outcomes:
      return "no contests"
    elif 3 in outcomes:
      return "draw"

    for fighter in data:
      outcome = fighter.get('Outcome', {})
      if outcome.get('OutcomeId') == 1:
        name = fighter.get('Name', {})
        first = name.get('FirstName', '')
        last = name.get('LastName', '')
        return f"{first} {last}".strip()

    return None

  fights_list = []

  # Descargar y analizar el código HTML de la página
  html = requests.get(url, headers=headers)
  soup = BeautifulSoup(html.content, 'html.parser')

  # Buscar las IDs de cada pelea
  tags = soup.find_all('div', class_='c-listing-fight')

  if not tags:
    return None

  for tag in tags:

    tag_id = tag.get('data-fmid')
    if not tag_id:
      continue

    # Definir url para la petición de cada pelea
    fights_url = f"https://d29dxerjsp82wz.cloudfront.net/api/v3/fight/live/{tag_id}.json"

    # Obtener información de la pelea
    res = requests.get(fights_url, headers=headers)
    data = res.json()

    data = data.get('LiveFightDetail', {})

    # Separar información
    weigh_class = data.get('WeightClass', {})
    result = data.get('Result', {})
    corner_info = data.get('Fighters', [])
    corner_stats = data.get('FightStats', [])

    if len(corner_info) == 2:
      red_data, blue_data = corner_info[0], corner_info[1]
    else:
      red_data, blue_data = {}, {}

    if len(corner_stats) == 2:
      red_stats, blue_stats = corner_stats[0], corner_stats[1]
    else:
      red_stats, blue_stats = {}, {}

    # Completar informción de la pelea
    fight = {
        'event': data.get('Event', {}).get('Name', None),
        'weight_class': "CatchWeight" if weigh_class.get('CatchWeight') else weigh_class.get('Description', None),  # Fight info
        'weigh': weigh_class.get('Weight', None),
        'accolades': (data.get('Accolades', [{}])[0].get('Type', 'Regular') if data.get('Accolades') else 'Regular'),
        'possible_rounds': data.get('RuleSet', {}).get('PossibleRounds', None),
        'card_segment': data.get('CardSegment', None),
        'fight_order': data.get('FightOrder', None),
        'status': data.get('Status', None),
        'winner': get_outcome(data.get('Fighters', [])),
        'round': result.get('EndingRound', None),
        'time': result.get('EndingTime', None),
        'method': result.get('Method', None),
        'ending_submission': result.get('EndingSubmission', None),
        'fight_of_the_night': result.get('FightOfTheNight', None),
        'referee': f"{data.get('Referee', {}).get('FirstName', '')} {data.get('Referee', {}).get('LastName', '')}".strip(),
        'red_corner': f"{red_data.get('Name', {}).get('FirstName', '')} {red_data.get('Name', {}).get('LastName', '')}".strip(),  # Red Corner info
        'total_strikes_attempted_red': red_stats.get('TotalStrikesAttempted', None),
        'total_strikes_landed_red': red_stats.get('TotalStrikesLanded', None),
        'significant_strikes_attempted_red': red_stats.get('SigStrikesAttempted', None),
        'significant_strikes_landed_red': red_stats.get('SigStrikesLanded', None),
        'knockdowns_red': red_stats.get('Knockdowns', None),
        'head_significant_strikes_red': red_stats.get('SigHeadStrikesLanded', None),  # By Target
        'body_significant_strikes_red': red_stats.get('SigBodyStrikesLanded', None),
        'leg_significant_strikes_red': red_stats.get('SigLegStrikesLanded', None),
        'distance_significant_strikes_red': red_stats.get('SigDistanceStrikesLanded', None),  # By Position
        'clinch_significant_strikes_red': red_stats.get('SigClinchStrikesLanded', None),
        'ground_significant_strikes_red': red_stats.get('SigGroundStrikesLanded', None),
        'takedowns_attempted_red': red_stats.get('TakedownsAttempted', None),  # Takedowns
        'takedowns_landed_red': red_stats.get('TakedownsLanded', None),
        'submission_attempts_red': red_stats.get('SubmissionsAttempted', None),
        'reversals_red': red_stats.get('Reversals', None),  # Control
        'clinch_control_time_red': red_stats.get('ClinchControlTime', None),
        'ground_control_time_red': red_stats.get('GroundControlTime', None),
        'ko_of_the_night_red': red_data.get('KOOfTheNight', None),  # Awards
        'submission_of_the_night_red': red_data.get('SubmissionOfTheNight', None),
        'performance_of_the_night_red': red_data.get('PerformanceOfTheNight', None),
        'blue_corner': f"{blue_data.get('Name', {}).get('FirstName', '')} {blue_data.get('Name', {}).get('LastName', '')}".strip(),  # Blue Corner info
        'total_strikes_attempted_blue': blue_stats.get('TotalStrikesAttempted', None),
        'total_strikes_landed_blue': blue_stats.get('TotalStrikesLanded', None),
        'significant_strikes_attempted_blue': blue_stats.get('SigStrikesAttempted', None),
        'significant_strikes_landed_blue': blue_stats.get('SigStrikesLanded', None),
        'knockdowns_blue': blue_stats.get('Knockdowns', None),
        'head_significant_strikes_blue': blue_stats.get('SigHeadStrikesLanded', None),  # By Target
        'body_significant_strikes_blue': blue_stats.get('SigBodyStrikesLanded', None),
        'leg_significant_strikes_blue': blue_stats.get('SigLegStrikesLanded', None),
        'distance_significant_strikes_blue': blue_stats.get('SigDistanceStrikesLanded', None),  # By Position
        'clinch_significant_strikes_blue': blue_stats.get('SigClinchStrikesLanded', None),
        'ground_significant_strikes_blue': blue_stats.get('SigGroundStrikesLanded', None),
        'takedowns_attempted_blue': blue_stats.get('TakedownsAttempted', None),  # Takedowns
        'takedowns_landed_blue': blue_stats.get('TakedownsLanded', None),
        'submission_attempts_blue': blue_stats.get('SubmissionsAttempted', None),
        'reversals_blue': blue_stats.get('Reversals', None),  # control
        'clinch_control_time_blue': blue_stats.get('ClinchControlTime', None),
        'ground_control_time_blue': blue_stats.get('GroundControlTime', None),
        'ko_of_the_night_blue': blue_data.get('KOOfTheNight', None),  # Awards
        'submission_of_the_night_blue': blue_data.get('SubmissionOfTheNight', None),
        'performance_of_the_night_blue': blue_data.get('PerformanceOfTheNight', None)
    }

    fights_list.append(fight)

  return fights_list

### Events scraper

In [ ]:
def ufc_events_scraper(url, headers, cookies, params):
  fights_list = []
  events_list = []
  count = 0

  while True:

    # Descargar y convertir la respuesta JSON de la página
    response = requests.get(url, headers=headers, cookies=cookies, params=params)
    data = response.json()

    # Extraer contenido HTML
    html = None
    for item in data:
      if item.get('command') == 'insert' and item.get('data'):
        html = item['data']
        break

    # Analizar  HTML y obtener enlaces
    soup = BeautifulSoup(html, 'html.parser')
    href_in_page = soup.select('h3.c-card-event--result__headline a')

    if not href_in_page:
      break

    # Obtener y guardar información de las peleas y el evento
    for a_tag in href_in_page:
      event_fights = get_event_fights("https://www.ufc.com" + a_tag['href'], config['events']['headers'])
      event = get_event("https://www.ufc.com" + a_tag['href'], config['events']['headers'])
      if event_fights is not None:
        fights_list.extend(event_fights)
      if event is not None:
        events_list.append(event)
      count += 1


    # Imprimir estado del progreso de la ejecución
    print(f"Registrados {len(events_list)} eventos")
    clear_output(wait=True)

    params['page'] += 1
    time.sleep(1)

  return pd.DataFrame(fights_list), pd.DataFrame(events_list)

# Crear csv

In [ ]:
today_date = date.today().strftime("%d_%m_%Y")

In [ ]:
profiles_file = f'ufc_fighters_profiles_{today_date}.csv'
profiles_path = f'/content/drive/MyDrive/ufc_scraper/{profiles_file}'

try:
  fighters = pd.read_csv(profiles_path)
  print("Cargado desde archivo")

except FileNotFoundError:
  fighters = ufc_fighters_scraper(config['fighters']['url'], headers=config['fighters']['headers'], cookies=config['fighters']['cookies'], params=config['fighters']['params'])
  fighters.to_csv(profiles_path, index=False)
  print("Guardado en archivo")

Cargado desde archivo


In [ ]:
fights_file = f'ufc_fights_{today_date}.csv'
events_file = f'ufc_events_{today_date}.csv'

fights_path = f'/content/drive/MyDrive/ufc_scraper/{fights_file}'
events_path = f'/content/drive/MyDrive/ufc_scraper/{events_file}'

try:
  fights = pd.read_csv(fights_path)
  print("Información de peleas cargada desde archivo")

  events = pd.read_csv(events_path)
  print("Información de eventos cargada desde archivo")

except FileNotFoundError:
  fights, events = ufc_events_scraper(config['events']['url'], headers=config['events']['headers'], cookies=config['events']['cookies'], params=config['events']['params'])
  fights.to_csv(fights_path, index=False)
  events.to_csv(events_path, index=False)
  print("Información de eventos y peleas guardadas en archivo")

Información de peleas cargada desde archivo
Información de eventos cargada desde archivo


In [ ]:
fighters

,name,nickname,place_of_birth,age,height,weight,reach,leg_reach,fighting_style,ufc_debut_date,...,significant_strikes_attempted,strikes_by_standing,strikes_by_clinch,strikes_by_ground,strikes_to_the_head,strikes_to_the_body,strikes_to_the_leg,takedowns_landed,takedowns_attempted,is_active
0,Danny Abbadi,"""The Assassin""","Orlando, United States",39.0,71.0,156.0,NaN,NaN,NaN,"Jun. 24, 2006",...,155.0,42.0,14.0,3.0,51.0,NaN,0.0,NaN,NaN,True
1,Nariman Abbassov,NaN,Kazakhstan,29.0,NaN,156.0,NaN,NaN,NaN,"Sep. 7, 2025",...,225.0,45.0,0.0,0.0,26.0,NaN,11.0,NaN,2.0,True
2,Tank Abbott,"""Tank""","Huntington Beach, United States",NaN,72.0,253.0,NaN,NaN,NaN,"Jul. 14, 1995",...,63.0,5.0,11.0,3.0,17.0,NaN,0.0,NaN,3.0,True
3,Hamdy Abdelwahab,"""The Hammer""",Egypt,32.0,74.0,264.5,72.0,41.0,Wrestler,"Jul. 30, 2022",...,320.0,125.0,2.0,30.0,110.0,NaN,29.0,3.0,6.0,True
4,Mansur Abdul-Malik,NaN,"Pittsburgh, United States",27.0,74.0,186.0,79.5,43.0,Freestyle,"Nov. 9, 2024",...,268.0,52.0,14.0,65.0,119.0,NaN,6.0,NaN,3.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,Farès Ziam,"""Smile Killer""","Vénissieux, Auvergne-Rhône-Alpes, France",27.0,73.0,156.0,75.0,42.0,Kickboxer,"Sep. 7, 2019",...,661.0,241.0,42.0,59.0,223.0,NaN,52.0,NaN,32.0,True
3060,James Zikic,"""The Messiah""","Watford, United Kingdom",46.0,74.0,202.0,NaN,NaN,NaN,"Jul. 13, 2002",...,125.0,18.0,16.0,10.0,24.0,NaN,9.0,NaN,4.0,True
3061,Cat Zingano,"""Alpha""","Winona, United States",40.0,66.0,144.0,68.0,38.0,NaN,"Apr. 14, 2013",...,295.0,66.0,44.0,71.0,107.0,NaN,28.0,13.0,20.0,True
3062,Igor Zinoviev,NaN,United States,NaN,0.0,0.0,NaN,NaN,NaN,"Mar. 13, 1998",...,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,True


In [ ]:
fights

,event,weight_class,weigh,accolades,possible_rounds,card_segment,fight_order,status,winner,round,...,ground_significant_strikes_blue,takedowns_attempted_blue,takedowns_landed_blue,submission_attempts_blue,reversals_blue,clinch_control_time_blue,ground_control_time_blue,ko_of_the_night_blue,submission_of_the_night_blue,performance_of_the_night_blue
0,UFC Fight Night: Imavov vs. Borralho,Middleweight,171-185,Regular,5.0,Main,1.0,Final,Nassourdine Imavov,5.0,...,0.0,5.0,0.0,0.0,0.0,1:20,0:00,False,False,False
1,UFC Fight Night: Imavov vs. Borralho,Lightweight,146-155,Regular,3.0,Main,2.0,Final,Benoit Saint Denis,2.0,...,0.0,0.0,0.0,0.0,0.0,0:06,0:28,False,False,False
2,UFC Fight Night: Imavov vs. Borralho,Light Heavyweight,186-205,Regular,3.0,Main,3.0,Final,Modestas Bukauskas,1.0,...,0.0,2.0,0.0,0.0,0.0,0:45,0:00,False,False,False
3,UFC Fight Night: Imavov vs. Borralho,Lightweight,146-155,Regular,3.0,Main,4.0,Final,Mason Jones,2.0,...,32.0,2.0,2.0,1.0,0.0,0:04,3:37,False,False,False
4,UFC Fight Night: Imavov vs. Borralho,Welterweight,156-170,Regular,3.0,Main,5.0,Final,Axel Sola,3.0,...,0.0,0.0,0.0,0.0,0.0,0:00,0:00,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8350,UFC 1: The Beginning,Open Weight,NaN,Regular,1.0,Main,6.0,Final,Royce Gracie,1.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,False,False,False
8351,UFC 1: The Beginning,Open Weight,NaN,Regular,1.0,Main,8.0,Final,Gerard Gordeau,1.0,...,0.0,1.0,0.0,0.0,0.0,NaN,NaN,False,False,False
8352,UFC 1: The Beginning,Open Weight,NaN,Regular,1.0,Main,5.0,Final,Ken Shamrock,1.0,...,2.0,0.0,0.0,0.0,0.0,NaN,NaN,False,False,False
8353,UFC 1: The Beginning,Open Weight,NaN,Regular,1.0,Main,7.0,Final,Kevin Rosier,1.0,...,2.0,0.0,0.0,0.0,0.0,NaN,NaN,False,False,False


In [ ]:
events.head()

,name,date,time_zone,country,state,city,venue
0,UFC Fight Night: Imavov vs. Borralho,2025-09-06T16:00Z,GMT+02:00,France,Ile-de-France,Paris,Accor Arena
1,UFC Fight Night: Walker vs. Zhang,2025-08-23T07:00Z,GMT+08:00,China,Hebei,Shanghai,Shanghai Indoor Arena
2,UFC 319: Du Plessis vs. Chimaev,2025-08-16T22:30Z,GMT-05:00,USA,Illinois,Chicago,United Center
3,UFC Fight Night: Dolidze vs. Hernandez,2025-08-09T20:00Z,GMT-07:00,USA,Nevada,Las Vegas,UFC Apex
4,UFC Fight Night: Taira vs. Park,2025-08-02T22:00Z,GMT-07:00,USA,Nevada,Las Vegas,UFC Apex
